In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Data Preparation

In [4]:
breast_cancer_data = datasets.load_breast_cancer()
X, y = breast_cancer_data.data, breast_cancer_data.target
print(X.shape, y.shape)

n_samples, n_features = X.shape

(569, 30) (569,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [13]:
X_train_torch = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_torch = torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_torch = torch.from_numpy(y_train.astype(np.float32))
y_test_torch = torch.from_numpy(y_test.astype(np.float32))

y_train_torch = y_train_torch.view(y_train_torch.shape[0], 1)
y_test_torch = y_test_torch.view(y_test_torch.shape[0], 1)

# Define Model

In [73]:
# f = w*x + b, then pass through a sigmoid function

class LogisticRegression(nn.Module):
    def __init__(self, n_features):
        super(LogisticRegression, self).__init__()
        
        # define layers
        self.lin = nn.Linear(in_features=n_features, out_features=1)
    
    def forward(self, X):
        y_pred = torch.sigmoid(self.lin(X))
        return y_pred

model = LogisticRegression(n_features=n_features)
criterion = nn.BCELoss() # binary cross entropy loss
lr = 0.1
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, weight_decay=0.1)
n_epoch = 1000

In [74]:
for epoch in range(n_epoch):
    y_pred = model(X_train_torch)
    loss_val = criterion(y_pred, y_train_torch)
    loss_val.backward()
    optimizer.step()
    optimizer.zero_grad()

    with torch.no_grad():
        y_pred_cls = y_pred.round()
        acc = y_pred_cls.eq(y_train_torch).sum() / float(y_train_torch.shape[0])
        test_prod = model(X_test_torch)
        test_pred = test_prod.round() # 1 if >= 0.5 else 0
        test_acc = test_pred.eq(y_test_torch).sum() / float(y_test_torch.shape[0])

    if (epoch+1) % 100 == 0:
        print(f'epoch {epoch+1}: Loss={loss_val.item():.4f}. Train Acc={acc:.4f}. Test accuracy = {test_acc:.4f}')

with torch.no_grad():
    test_prod = model(X_test_torch)
    test_pred = test_prod.round() # 1 if >= 0.5 else 0
    acc = test_pred.eq(y_test_torch).sum() / float(y_test_torch.shape[0])
    print(f'Test accuracy = {acc:.4f}')




epoch 100: Loss=0.1330. Train Acc=0.9824. Test accuracy = 0.9649
epoch 200: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 300: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 400: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 500: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 600: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 700: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 800: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 900: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
epoch 1000: Loss=0.1335. Train Acc=0.9824. Test accuracy = 0.9649
Test accuracy = 0.9649
